# Salaries

https://machinelearningmastery.com/stacking-ensemble-machine-learning-with-python/


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

ds = pd.read_csv('ds_salaries.csv')
ds = ds.drop(['salary'], axis = 1)
# we don't have any nulls 
print(ds.columns)

# NLP:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')



Index(['Unnamed: 0', 'work_year', 'experience_level', 'employment_type',
       'job_title', 'salary_currency', 'salary_in_usd', 'employee_residence',
       'remote_ratio', 'company_location', 'company_size'],
      dtype='object')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shahafdan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
ds['job_title'] = [tokenizer.tokenize(x.lower()) for x in ds['job_title']]
ds['job_title'] = ds['job_title'].apply(lambda x: [w for w in x if not w in stop_words])
ds['job_title'] = [' '.join(x) for x in ds['job_title']]

ds['salary_currency'] = pd.factorize(ds['salary_currency'])[0]
ds['experience_level'] = pd.factorize(ds['experience_level'])[0]
ds['employee_residence'] = pd.factorize(ds['employee_residence'])[0]
ds['company_location'] = pd.factorize(ds['company_location'])[0]
ds['company_size'] = pd.factorize(ds['company_size'])[0]
ds['employment_type'] = pd.factorize(ds['employment_type'])[0]
ds['remote_ratio'] = pd.factorize(ds['remote_ratio'])[0]

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler


# Set X and y
X = ds[['salary_currency', 'experience_level', 'employee_residence', 'company_size', 'company_location', 'employment_type', 'remote_ratio']]
y = ds['salary_in_usd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
model = RandomForestRegressor()
model.fit(X_train, y_train)
rf_predictions = model.predict(X_test)
rf_score = model.score(X_test, y_test)

model = LinearRegression()
model.fit(X_train, y_train)
linreg_predictions = model.predict(X_test)
linreg_score = model.score(X_test, y_test)

model = KNeighborsRegressor()
model.fit(X_train, y_train)
knn_pred = model.predict(X_test)
knn_score = model.score(X_test, y_test)

pred_test = pd.DataFrame({
    'test': np.array(y_test), 
    'rf_predictions': list(np.array(rf_predictions)), 
    'knn_predictions': list(np.array(knn_pred)), 
    'svr_predictions': list(np.array(svr_pred)), 
    'linreg_predictions': list(np.array(linreg_predictions))}, 
    columns=['test', 'rf_predictions', 'knn_predictions', 'svr_predictions', 'linreg_predictions'])
pred_test = pred_test.sort_values(by = 'test')
ind = range(len(pred_test))

plt.figure(figsize = (25, 6))
plt.title(model.score(X_test, y_test))
plt.plot(ind, pred_test['test'], linewidth = 2, color = 'orange')
plt.plot(ind, pred_test['rf_predictions'], linewidth = 2, color = 'blue', label = "Random Forest: " + str(round(100 * rf_score, 2)) + "% accuracy")
plt.plot(ind, pred_test['linreg_predictions'], linewidth = 2, color = 'green', label = "Linear Regression: " + str(round(100 * linreg_score, 2)) + "% accuracy")
plt.plot(ind, pred_test['knn_predictions'], linewidth = 2, color = 'red', label = "KNN Regressor: " + str(round(100 * knn_score, 2)) + "% accuracy")
plt.plot(ind, pred_test['svr_predictions'], linewidth = 2, color = 'pink', label = "SVR Regressor: " + str(round(100 * svr_score, 2)) + "% accuracy")
plt.legend()
plt.show()




X = ds[['job_title', 'salary_currency', 'experience_level', 'employee_residence', 'company_size', 'company_location', 'employment_type', 'remote_ratio']]
y = ds['salary_in_usd']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

vectorizer = TfidfVectorizer()
column_transformer = ColumnTransformer([('tfidf', vectorizer, 'job_title')], remainder = 'passthrough')
tfidf = vectorizer.fit_transform(X_train)
model = LinearRegression()
pipe = Pipeline([('tfidf', column_transformer), ('classify', model)])
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)
# print(X_train)
# predictions = pipe.predict(X_test, y_test)

# pred_test = pd.DataFrame({'test': np.array(y_test), 'predictions': list(np.array(predictions))}, columns=['test', 'predictions'])
# pred_test = pred_test.sort_values(by = 'test')
# ind = range(len(pred_test))

# plt.figure(figsize = (25, 15))
# plt.plot(ind, pred_test['test'])
# plt.plot(ind, pred_test['predictions'])
# plt.show()

ValueError: Expected 2D array, got 1D array instead:
array=[125000.  88654. 144854. 152500.  38400.  65438.  99360. 150000. 180000.
 153667. 147000. 140000. 135000. 160080. 170000.  53192.  19609. 211500.
 105000.   8000.  13400.  90320. 170000.  86703.  40189. 130000.  45807.
 450000.  79833.  67000. 130000.  61300. 132320. 102100.  72212.  47899.
 154600.  50000.  80000. 123000.  66022.   5707.  32974. 140000. 100000.
  69999. 100000. 130000. 160000. 380000.  45807. 184700. 162674. 250000.
  60000. 150000.  76958. 276000. 160000. 150000.  54000. 200000. 230000.
  98158.  99000.  81666.  88654. 209100.  65949. 173762. 200000.  91237.
  18907. 180000.  99000.  65438.  90000. 100000. 160000.  10354. 104702.
 220000.   4000. 135000.  62651.  96282.  25532. 115000.  80000. 123000.
 205300.  48000.  45391.  50000. 240000.  93700.  35735. 135000.  56000.
  60000. 210000.  38776. 146000. 209100.  87932. 130000. 220110.  70139.
  98158. 325000.  21983.  46597. 161342. 136620.  50180. 159000. 